# Network Analysis for ABCD data

http://dx.plos.org/10.1371/journal.pbio.1002328  
https://www.sciencedirect.com/science/article/pii/S105381191730109X?via%3Dihub

In [ ]:
# ! pip install python-louvain
# ! pip install pandas==0.24.1 --user

In [75]:
import glob
import os
import networkx as nx
import numpy as np
import pandas as pd
import community
from sklearn.metrics.cluster import normalized_mutual_info_score
import bz2
import pickle
import pdb
import statistics
import matplotlib
matplotlib.use("Qt5Agg")
import matplotlib.pyplot as plt

from visbrain.objects import ConnectObj, SceneObj, SourceObj, BrainObj
from visbrain.io import download_file

import bct
import json

ModuleNotFoundError: No module named 'numpy_lru_cache_decorator'

## Data of interest (from R notebook)

### Read in labels

In [2]:
labels = pd.read_csv('/Users/gracer/Google Drive/ABCD/important_txt/locations.csv', sep=",")

In [3]:
interest =  pd.read_csv('/Users/gracer/Google Drive/ABCD/important_txt/data4analysis.txt', sep=" ", header=None, 
                 index_col=False)
interest.head()

,0,1,2,3
0,sub-NDARINVMFVA1BZF,F,latepubertal,Obese
1,sub-NDARINV019DXLU4,F,latepubertal,Obese
2,sub-NDARINVT472A3DT,F,latepubertal,Obese
3,sub-NDARINV4D917H14,F,latepubertal,Obese
4,sub-NDARINVD9L81NY5,F,latepubertal,Obese


In [4]:
subs=interest[0]
subs[1:10]

1    sub-NDARINV019DXLU4
2    sub-NDARINVT472A3DT
3    sub-NDARINV4D917H14
4    sub-NDARINVD9L81NY5
5    sub-NDARINV8PHEF3UH
6    sub-NDARINV08FUB58A
7    sub-NDARINV8JTY86NW
8    sub-NDARINV1MKWVMP1
9    sub-NDARINVK1GY1F0R
Name: 0, dtype: object

## Get data

In [5]:
data = glob.glob('/Users/gracer/Google Drive/ABCD/ABCD_puberty/_data/ABCDworking/sub-*/keep/sub-NDAR*_ses-baselineYear1Arm1_task-rest_run-0*_bold_brain_norm_r_matrix.csv')






In [6]:
data[0:10]

['/Users/gracer/Google Drive/ABCD/ABCD_puberty/_data/ABCDworking/sub-NDARINVKCU1HRMA/keep/sub-NDARINVKCU1HRMA_ses-baselineYear1Arm1_task-rest_run-01_bold_brain_norm_r_matrix.csv',
 '/Users/gracer/Google Drive/ABCD/ABCD_puberty/_data/ABCDworking/sub-NDARINVKCU1HRMA/keep/sub-NDARINVKCU1HRMA_ses-baselineYear1Arm1_task-rest_run-02_bold_brain_norm_r_matrix.csv',
 '/Users/gracer/Google Drive/ABCD/ABCD_puberty/_data/ABCDworking/sub-NDARINV20CNYBBC/keep/sub-NDARINV20CNYBBC_ses-baselineYear1Arm1_task-rest_run-01_bold_brain_norm_r_matrix.csv',
 '/Users/gracer/Google Drive/ABCD/ABCD_puberty/_data/ABCDworking/sub-NDARINV20CNYBBC/keep/sub-NDARINV20CNYBBC_ses-baselineYear1Arm1_task-rest_run-02_bold_brain_norm_r_matrix.csv',
 '/Users/gracer/Google Drive/ABCD/ABCD_puberty/_data/ABCDworking/sub-NDARINV20CNYBBC/keep/sub-NDARINV20CNYBBC_ses-baselineYear1Arm1_task-rest_run-03_bold_brain_norm_r_matrix.csv',
 '/Users/gracer/Google Drive/ABCD/ABCD_puberty/_data/ABCDworking/sub-NDARINVJ67B8F0J/keep/sub-NDARIN

#### Make a dictonary to store the path and subject ID

In [7]:
my_dict={}
for item in data:
    name=item.split('/')[8]
    print(name)
    my_dict.setdefault(name, []).append(item)

sub-NDARINVKCU1HRMA
sub-NDARINVKCU1HRMA
sub-NDARINV20CNYBBC
sub-NDARINV20CNYBBC
sub-NDARINV20CNYBBC
sub-NDARINVJ67B8F0J
sub-NDARINVJ67B8F0J
sub-NDARINVJ67B8F0J
sub-NDARINVJ67B8F0J
sub-NDARINVH8HNJZD4
sub-NDARINVH8HNJZD4
sub-NDARINVH8HNJZD4
sub-NDARINVH8HNJZD4
sub-NDARINV6PX8VT24
sub-NDARINV6PX8VT24
sub-NDARINV6PX8VT24
sub-NDARINV6PX8VT24
sub-NDARINVHNPRTMWZ
sub-NDARINVHNPRTMWZ
sub-NDARINVHNPRTMWZ
sub-NDARINVHNPRTMWZ
sub-NDARINV78TL163B
sub-NDARINV78TL163B
sub-NDARINV78TL163B
sub-NDARINV2GDYKWG2
sub-NDARINV2GDYKWG2
sub-NDARINV8JTY86NW
sub-NDARINV8JTY86NW
sub-NDARINV8JTY86NW
sub-NDARINVXNEVU6X0
sub-NDARINVXNEVU6X0
sub-NDARINVXNEVU6X0
sub-NDARINVXNEVU6X0
sub-NDARINV63JMZ9HT
sub-NDARINV63JMZ9HT
sub-NDARINV63JMZ9HT
sub-NDARINV63JMZ9HT
sub-NDARINVKVV9NU2R
sub-NDARINVKVV9NU2R
sub-NDARINVKVV9NU2R
sub-NDARINVKVV9NU2R
sub-NDARINV190GWDVP
sub-NDARINV190GWDVP
sub-NDARINV190GWDVP
sub-NDARINV190GWDVP
sub-NDARINV53DLM6RN
sub-NDARINV53DLM6RN
sub-NDARINV53DLM6RN
sub-NDARINV53DLM6RN
sub-NDARINV23XZD51D


sub-NDARINV6VT5M0AL
sub-NDARINV6VT5M0AL
sub-NDARINVGEUD7WNN
sub-NDARINVGEUD7WNN
sub-NDARINVGEUD7WNN
sub-NDARINVGEUD7WNN
sub-NDARINVL4V0U23M
sub-NDARINVL4V0U23M
sub-NDARINVL4V0U23M
sub-NDARINVL4V0U23M
sub-NDARINV2LY3HDP4
sub-NDARINV2LY3HDP4
sub-NDARINV2LY3HDP4
sub-NDARINV2LY3HDP4
sub-NDARINV1KWBW4RD
sub-NDARINV1KWBW4RD
sub-NDARINV1KWBW4RD
sub-NDARINV1KWBW4RD
sub-NDARINVHPT4RWK5
sub-NDARINVHPT4RWK5
sub-NDARINVHPT4RWK5
sub-NDARINVHPT4RWK5
sub-NDARINV2ERWUABP
sub-NDARINV2ERWUABP
sub-NDARINV2ERWUABP
sub-NDARINV2ERWUABP
sub-NDARINVH6H319AA
sub-NDARINVH6H319AA
sub-NDARINVH6H319AA
sub-NDARINVH6H319AA
sub-NDARINVG3C90H0L
sub-NDARINVG3C90H0L
sub-NDARINVG3C90H0L
sub-NDARINVG3C90H0L
sub-NDARINVL8MTLH03
sub-NDARINVL8MTLH03
sub-NDARINVL8MTLH03
sub-NDARINVL8MTLH03
sub-NDARINV36DKMERA
sub-NDARINV36DKMERA
sub-NDARINV36DKMERA
sub-NDARINV36DKMERA
sub-NDARINV5ZAP9LUZ
sub-NDARINV26YB6ZJ9
sub-NDARINV26YB6ZJ9
sub-NDARINV26YB6ZJ9
sub-NDARINV26YB6ZJ9
sub-NDARINV2NT59PWC
sub-NDARINV2NT59PWC
sub-NDARINV2NT59PWC


sub-NDARINV67G76XVR
sub-NDARINV67G76XVR
sub-NDARINV67G76XVR
sub-NDARINVJTHY8P5B
sub-NDARINVJTHY8P5B
sub-NDARINVJTHY8P5B
sub-NDARINVJTHY8P5B
sub-NDARINV6JJBMHZL
sub-NDARINV6JJBMHZL
sub-NDARINV6JJBMHZL
sub-NDARINV6JJBMHZL
sub-NDARINVH4DCU3R1
sub-NDARINVH4DCU3R1
sub-NDARINVH4DCU3R1
sub-NDARINVH4DCU3R1
sub-NDARINVT472A3DT
sub-NDARINVT472A3DT
sub-NDARINVT472A3DT
sub-NDARINVT472A3DT
sub-NDARINV3HC4V43B
sub-NDARINV3HC4V43B
sub-NDARINV3HC4V43B
sub-NDARINV3HC4V43B
sub-NDARINVK0LXC0U4
sub-NDARINVK0LXC0U4
sub-NDARINVK0LXC0U4
sub-NDARINVK0LXC0U4
sub-NDARINV3WRKC6EJ
sub-NDARINV3WRKC6EJ
sub-NDARINV3WRKC6EJ
sub-NDARINV3WRKC6EJ
sub-NDARINV5KKUBDAY
sub-NDARINV5KKUBDAY
sub-NDARINV5KKUBDAY
sub-NDARINV0YE7L9KU
sub-NDARINV0YE7L9KU
sub-NDARINV0YE7L9KU
sub-NDARINV0YE7L9KU
sub-NDARINVL02ZXJAE
sub-NDARINVL02ZXJAE
sub-NDARINVL02ZXJAE
sub-NDARINVL02ZXJAE
sub-NDARINVH93P807E
sub-NDARINVH93P807E
sub-NDARINVH93P807E
sub-NDARINVH93P807E
sub-NDARINV2Z2HJFG1
sub-NDARINV2Z2HJFG1
sub-NDARINV2Z2HJFG1
sub-NDARINV6EJF2KCZ


### Make a dictonary of all the file paths common to the subject list 

In [8]:
path_dict = {k: my_dict[k] for k in subs if k in my_dict}
print(my_dict)

{'sub-NDARINVKCU1HRMA': ['/Users/gracer/Google Drive/ABCD/ABCD_puberty/_data/ABCDworking/sub-NDARINVKCU1HRMA/keep/sub-NDARINVKCU1HRMA_ses-baselineYear1Arm1_task-rest_run-01_bold_brain_norm_r_matrix.csv', '/Users/gracer/Google Drive/ABCD/ABCD_puberty/_data/ABCDworking/sub-NDARINVKCU1HRMA/keep/sub-NDARINVKCU1HRMA_ses-baselineYear1Arm1_task-rest_run-02_bold_brain_norm_r_matrix.csv'], 'sub-NDARINV20CNYBBC': ['/Users/gracer/Google Drive/ABCD/ABCD_puberty/_data/ABCDworking/sub-NDARINV20CNYBBC/keep/sub-NDARINV20CNYBBC_ses-baselineYear1Arm1_task-rest_run-01_bold_brain_norm_r_matrix.csv', '/Users/gracer/Google Drive/ABCD/ABCD_puberty/_data/ABCDworking/sub-NDARINV20CNYBBC/keep/sub-NDARINV20CNYBBC_ses-baselineYear1Arm1_task-rest_run-02_bold_brain_norm_r_matrix.csv', '/Users/gracer/Google Drive/ABCD/ABCD_puberty/_data/ABCDworking/sub-NDARINV20CNYBBC/keep/sub-NDARINV20CNYBBC_ses-baselineYear1Arm1_task-rest_run-03_bold_brain_norm_r_matrix.csv'], 'sub-NDARINVJ67B8F0J': ['/Users/gracer/Google Drive/AB

### Make a dictonary to read in the files

In [9]:
data_dict={}
num=[]

for key, value in path_dict.items():
    num.append(len(value))
    for i in value:
        x=pd.read_csv(i, header=None,index_col=False)
#         data_dict.setdefault(key, x)
        data_dict.setdefault(key, []).append(pd.read_csv(i, header=None,index_col=False))
#         data_dict[key]= pd.read_csv(i, header=None,index_col=False)

In [10]:
np.mean(num)

3.783333333333333

In [11]:
len(list(data_dict.keys()))

120

## Function to create dictionary of covariates

In [12]:
interest_dict=interest.set_index(interest[0]).to_dict()
def removekey(d, key):
    r = dict(d)
    del r[key]
    return r
cov_dict=removekey(interest_dict, 0)
cov_dict['sex'] = cov_dict.pop(1)
cov_dict['PCS'] = cov_dict.pop(2)
cov_dict['OVOB'] = cov_dict.pop(3)
# cov_dict['BMItile'] = cov_dict.pop(4)
# cov_dict['PDSscore'] = cov_dict.pop(5)
# cov_dict['age'] = cov_dict.pop(6)
print(cov_dict.keys())

dict_keys(['sex', 'PCS', 'OVOB'])


## Function to create graph objects

In [13]:
def create_corr_network_5(G, corr_direction, min_correlation):
    ##Creates a copy of the graph
    H = G.copy()
    
    ##Checks all the edges and removes some based on corr_direction
    for stock1, stock2, weight in list(G.edges(data=True)):
        ##if we only want to see the positive correlations we then delete the edges with weight smaller than 0        
        if corr_direction == "positive":
            ####it adds a minimum value for correlation. 
            ####If correlation weaker than the min, then it deletes the edge
            if weight["weight"] <0 or weight["weight"] < min_correlation:
                H.remove_edge(stock1, stock2)
        ##this part runs if the corr_direction is negative and removes edges with weights equal or largen than 0
        else:
            ####it adds a minimum value for correlation. 
            ####If correlation weaker than the min, then it deletes the edge
            if weight["weight"] >=0 or weight["weight"] > min_correlation:
                H.remove_edge(stock1, stock2)
    return(H)


### Function to make a graph object BY SUBJECT
This will return: 
* The edges (noramlized R correlation matrix, in pandas dataframe)
* The mean_FC (the mean functional connectivity per subject/node)
* The graphs (this will contain the raw graph object G as well as the the partion values from the modularity calculation)
* The mu is the mean of all the runs into a single correlation matrix per subject

In [14]:
def make_graphs(list_o_data, direction, min_cor):
    edge_dict={}
    FC_dict={}
    graph_dict={}
    
    print(len(list(list_o_data.keys())))
    j=0
    mylist=[]
    mu_network={}
    bad={}
    for key, val_list in list_o_data.items():
        print("on number %s"%(str(j)))
        j=j+1
        newlist=[]
        for item in val_list:
            
            print(np.array(item).diagonal())
            if np.all(np.array(item).diagonal()) == True:
                newlist.append(np.array(item))
                i=item.set_index(labels.ID)
                i.rename(columns=labels.ID, inplace=True)
                edge_dict.setdefault(key, []).append(i)
                
            else:
                print("%s is fucked"%key)
                bad.setdefault(key, []).append(item)
                
                
        try:        
            y=np.dstack(newlist)
            print(y.shape)
            y=np.rollaxis(y,-1)
            print(y.shape)
            mu=np.mean(y, axis=0)
            print(np.array(mu).diagonal())
            mu_network.setdefault(key, mu)

            m=x.mean()
            FC_dict.setdefault(key, []).append(m)

            G = nx.from_numpy_matrix(mu)
            for i, nlrow in labels.iterrows():
                G.node[i].update(nlrow[0:].to_dict())

            graph_dict.setdefault(key, []).append(G)

            partition = community.best_partition(create_corr_network_5(G, direction,min_cor))

            graph_dict.setdefault(key, []).append(partition)
        except ValueError:
            continue
        
        
#         pdb.set_trace()
              
    return({'edges':edge_dict, 'mean_FC':FC_dict, 'graphs':graph_dict, 'mu':mu_network})

In [15]:
GRAPHS=make_graphs(data_dict, "positive", 0)

120
on number 0
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 

on number 4
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1

on number 7
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1

on number 10
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 

on number 13
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 

...

on number 105
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.

on number 108
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 0. 0. 1. 0. 0. 1. 1. 0. 1. 1. 0. 0. 0. 0. 1. 1. 1. 0.
 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 0. 0. 0. 1. 0. 0. 0. 1. 1. 0. 1. 1. 0.
 0. 1. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 0. 1. 1. 0. 0. 0. 0. 0. 0. 1. 0.
 1. 1. 1. 0. 0. 0. 0. 1. 0. 1. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
sub-NDARINVL8XGD85G is fucked
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1.

on number 111
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.

on number 114
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.

on number 117
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.

In [16]:
len(list(GRAPHS['mu']))

104

In [17]:
for key in cov_dict.keys():
    for subkey, value in cov_dict[key].items():
        if subkey in GRAPHS['graphs']:
            GRAPHS['graphs'][subkey].append(value)
            print(GRAPHS['graphs'][subkey])

[<networkx.classes.graph.Graph object at 0x1c249a9240>, {0: 0, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 1, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0, 18: 0, 19: 0, 20: 0, 21: 0, 22: 0, 23: 0, 24: 0, 25: 0, 26: 0, 27: 0, 28: 0, 29: 0, 30: 0, 31: 2, 32: 0, 33: 3, 34: 2, 35: 0, 36: 0, 37: 0, 38: 0, 39: 0, 40: 2, 41: 0, 42: 0, 43: 3, 44: 2, 45: 3, 46: 3, 47: 0, 48: 2, 49: 2, 50: 2, 51: 2, 52: 3, 53: 1, 54: 3, 55: 2, 56: 0, 57: 3, 58: 3, 59: 2, 60: 2, 61: 3, 62: 3, 63: 3, 64: 3, 65: 3, 66: 3, 67: 3, 68: 0, 69: 0, 70: 2, 71: 0, 72: 2, 73: 0, 74: 3, 75: 0, 76: 3, 77: 3, 78: 0, 79: 0, 80: 0, 81: 3, 82: 0, 83: 1, 84: 1, 85: 1, 86: 1, 87: 0, 88: 1, 89: 1, 90: 1, 91: 0, 92: 1, 93: 2, 94: 2, 95: 3, 96: 3, 97: 3, 98: 0, 99: 2, 100: 1, 101: 0, 102: 2, 103: 3, 104: 3, 105: 2, 106: 3, 107: 0, 108: 0, 109: 2, 110: 3, 111: 3, 112: 3, 113: 3, 114: 2, 115: 0, 116: 2, 117: 3, 118: 3, 119: 3, 120: 0, 121: 0, 122: 0, 123: 0, 124: 0, 125: 3, 126: 3, 127: 3, 128: 3, 129: 3, 130: 3, 131: 

In [18]:
len(list(GRAPHS['graphs']))

104

In [19]:
cov_dict['PCS']['sub-NDARINV019DXLU4']

'latepubertal'

In [20]:
GRAPHS.keys()

dict_keys(['edges', 'mean_FC', 'graphs', 'mu'])

In [21]:
def merge_two_dicts(x, y):
    z = x.copy()   # start with x's keys and values
    z.update(y)    # modifies z with y's keys and values & returns None
    return z

In [22]:
z = merge_two_dicts(GRAPHS, interest_dict)

In [23]:
z.keys()

dict_keys(['edges', 'mean_FC', 'graphs', 'mu', 0, 1, 2, 3])

In [24]:
z['sex'] =z.pop(0)
z['sex'] =z.pop(1)
z['PCS'] =z.pop(2)
z['OVOB'] =z.pop(3)
# z['BMItile'] =z.pop(4)
# z['PDSscore'] =z.pop(5)
# z['age'] =z.pop(6)

### How to delete items in list that you accidentally made

In [25]:
# for key,value in GRAPHS['mean_FC'].items():
#      del value[-7:]

# Differences in modularity by subject

## Participation Coefficient
Partition networks into the modules, calculate the PC per node within each group. Higher PC indicates more distributed between network connectivity, while a PC of 0 signifies a node’s links are completely within its home network (within network).

#### Parameters
    ----------
    W : NxN np.ndarray
        binary/weighted directed/undirected connection matrix
    ci : Nx1 np.ndarray
        community affiliation vector
    degree : str
        Flag to describe nature of graph 'undirected': For undirected graphs
                                         'in': Uses the in-degree
                                         'out': Uses the out-degree

In [26]:
def participation_award(cor_mats, parts):
#     cor_mats need to be something like a dictionary of correlation matrices with the subject as the key
#     parts need to be the numerical modularity values
    allPC={}
    for keys, values in cor_mats.items():
        print(keys)
        cor_mat = np.array(values)
        test_array=np.array(list(list(z['graphs'][keys])[1].values()))
        testPART=np.vstack(test_array)

        PC=bct.participation_coef(W=cor_mat, ci= testPART)
        allPC[keys]=PC
        
    return(allPC)


In [27]:
allPC=participation_award(z['mu'],z['graphs'])

sub-NDARINVMFVA1BZF
sub-NDARINVD9L81NY5
sub-NDARINV8PHEF3UH
sub-NDARINV08FUB58A
sub-NDARINV8JTY86NW
sub-NDARINV1MKWVMP1
sub-NDARINVK1GY1F0R
sub-NDARINV4878W3DV
sub-NDARINVJZP468AD
sub-NDARINV56YBTJHF
sub-NDARINV2NVERUP6
sub-NDARINVKJKYNK5V
sub-NDARINV1Y1Z29D0
sub-NDARINVKTZGA2UH
sub-NDARINVH6K9F426
sub-NDARINVK8CYP94F
sub-NDARINV45HV5L52
sub-NDARINV72MV78KG
sub-NDARINVMA0MXL96
sub-NDARINV4EKBZJTW
sub-NDARINVUXT4TZJF
sub-NDARINV10ZLTFJZ
sub-NDARINV3XL11J4W
sub-NDARINV1T4GDEXG
sub-NDARINVL36MC5HJ
sub-NDARINV4HWVY2FD
sub-NDARINVKNRU5BYD
sub-NDARINV2GRXYP02
sub-NDARINVJNDY0DWK
sub-NDARINVJ6EUW017
sub-NDARINVGALZWY31
sub-NDARINV9AMTA922
sub-NDARINVAWG2NZC4
sub-NDARINVYRRPGLF4
sub-NDARINVX3PYJZPC
sub-NDARINVYZMY8DH5
sub-NDARINVLB1UUZT7
sub-NDARINV61VX090A
sub-NDARINV71N5JHR2
sub-NDARINVFV0JWJ60
sub-NDARINVH2N0P8GY
sub-NDARINV6UPZCK82
sub-NDARINVK0RLYCC7
sub-NDARINV415JRYRK
sub-NDARINV637CGUEV
sub-NDARINV5AC9TVN4
sub-NDARINV66708YNC
sub-NDARINVH0016MVZ
sub-NDARINVKFU0Z1FD
sub-NDARINVXU8K8M57


### Clustering Coefficient 
The weighted clustering coefficient is the average "intensity" of
    triangles around a node.
   #### Parameters
    ----------
    W : NxN np.ndarray
        weighted directed connection matrix
    Returns
    -------
    C : Nx1 np.ndarray
        clustering coefficient vector
    Notes
    -----
    Methodological note (also see clustering_coef_bd)
    The weighted modification is as follows:
    - The numerator: adjacency matrix is replaced with weights matrix ^ 1/3
    - The denominator: no changes from the binary version
    The above reduces to symmetric and/or binary versions of the clustering
    coefficient for respective graphs.

In [28]:
z.keys()

dict_keys(['edges', 'mean_FC', 'graphs', 'mu', 'sex', 'PCS', 'OVOB'])

In [29]:
def cluster_fuq(cor_mats):
    clusters={}
    for keys, values in cor_mats.items():
        CC=bct.clustering_coef_wd(values)
        clusters[keys]=CC
    return(clusters)
#         pdb.set_trace()

In [30]:
allCC=cluster_fuq(z['mu'])

### UR SO Random
randomization not working, here are each function with slight tweak in the null_model_und_sign function

In [ ]:
path=os.path.join("Users","⁨gracer","Google Drive","ABCD","⁨ABCD_puberty","⁨_data","currents")
print(path)

In [85]:
from functools import lru_cache, wraps
import numpy as np


def np_cache(*args, **kwargs):
    """LRU cache implementation for functions whose FIRST parameter is a numpy array

    >>> array = np.array([[1, 2, 3], [4, 5, 6]])

    >>> @np_cache(maxsize=256)
    ... def multiply(array, factor):
    ...     print("Calculating...")
    ...     return factor*array

    >>> multiply(array, 2)
    Calculating...
    array([[ 2,  4,  6],
           [ 8, 10, 12]])

    >>> multiply(array, 2)
    array([[ 2,  4,  6],
           [ 8, 10, 12]])

    >>> multiply.cache_info()
    CacheInfo(hits=1, misses=1, maxsize=256, currsize=1)
    
    """
    def decorator(function):
        @wraps(function)
        def wrapper(np_array, *args, **kwargs):
            hashable_array = array_to_tuple(np_array)
            return cached_wrapper(hashable_array, *args, **kwargs)

        @lru_cache(*args, **kwargs)
        def cached_wrapper(hashable_array, *args, **kwargs):
            array = np.array(hashable_array)
            return function(array, *args, **kwargs)

        def array_to_tuple(np_array):
            """Iterates recursivelly."""
            try:
                return tuple(array_to_tuple(_) for _ in np_array)
            except TypeError:
                return np_array

        # copy lru_cache attributes over too
        wrapper.cache_info = cached_wrapper.cache_info
        wrapper.cache_clear = cached_wrapper.cache_clear

        return wrapper

    return decorator

In [36]:
def get_rng(seed=None):
    """
    By default, or if `seed` is np.random, return the global RandomState
    instance used by np.random.
    If `seed` is a RandomState instance, return it unchanged.
    Otherwise, use the passed (hashable) argument to seed a new instance
    of RandomState and return it.
    Parameters
    ----------
    seed : hashable or np.random.RandomState or np.random, optional
    Returns
    -------
    np.random.RandomState
    """
    if seed is None or seed == np.random:
        return np.random.mtrand._rand
    elif isinstance(seed, np.random.RandomState):
        return seed
    try:
        rstate =  np.random.RandomState(seed)
    except ValueError:
        rstate = np.random.RandomState(random.Random(seed).randint(0, 2**32-1))
    return rstate

In [37]:
def pick_four_unique_nodes_quickly(n, seed=None):
    '''
    This is equivalent to np.random.choice(n, 4, replace=False)
    Another fellow suggested np.random.random_sample(n).argpartition(4) which is
    clever but still substantially slower.
    '''
    rng = get_rng(seed)
    k = rng.randint(n**4)
    a = k % n
    b = k // n % n
    c = k // n ** 2 % n
    d = k // n ** 3 % n
    if (a != b and a != c and a != d and b != c and b != d and c != d):
        return (a, b, c, d)
    else:
        # the probability of finding a wrong configuration is extremely low
        # unless for extremely small n. if n is extremely small the
        # computational demand is not a problem.

        # In my profiling it only took 0.4 seconds to include the uniqueness
        # check in 1 million runs of this function so I think it is OK.
        return pick_four_unique_nodes_quickly(n, rng)

In [38]:
def randmio_und_signed(R, itr, seed=None):
    '''
    This function randomizes an undirected weighted network with positive
    and negative weights, while simultaneously preserving the degree
    distribution of positive and negative weights. The function does not
    preserve the strength distribution in weighted networks.
    Parameters
    ----------
    W : NxN np.ndarray
        undirected binary/weighted connection matrix
    itr : int
        rewiring parameter. Each edge is rewired approximately itr times.
    seed : hashable, optional
        If None (default), use the np.random's global random state to generate random numbers.
        Otherwise, use a new np.random.RandomState instance seeded with the given value.
    Returns
    -------
    R : NxN np.ndarray
        randomized network
    '''
    rng = get_rng(seed)
    R = R.copy()
    n = len(R)

    itr *= int(n * (n -1) / 2)

    max_attempts = int(np.round(n / 2))
    eff = 0

    for it in range(int(itr)):
        att = 0
        while att <= max_attempts:

            a, b, c, d = pick_four_unique_nodes_quickly(n, rng)

            r0_ab = R[a, b]
            r0_cd = R[c, d]
            r0_ad = R[a, d]
            r0_cb = R[c, b]

            #rewiring condition
            if (    np.sign(r0_ab) == np.sign(r0_cd) and
                    np.sign(r0_ad) == np.sign(r0_cb) and
                    np.sign(r0_ab) != np.sign(r0_ad)):
        
                R[a, d] = R[d, a] = r0_ab
                R[a, b] = R[b, a] = r0_ad

                R[c, b] = R[b, c] = r0_cd
                R[c, d] = R[d, c] = r0_cb

                eff += 1
                break

            att += 1

    return R, eff

In [99]:
@np_cache()
def null_model_und_sign(W, bin_swaps=5, wei_freq=.1, seed=None):
    '''
    This function randomizes an undirected network with positive and
    negative weights, while preserving the degree and strength
    distributions. This function calls randmio_und.m
    Parameters
    ----------
    W : NxN np.ndarray
        undirected weighted connection matrix
    bin_swaps : int
        average number of swaps in each edge binary randomization. Default
        value is 5. 0 swaps implies no binary randomization.
    wei_freq : float
        frequency of weight sorting in weighted randomization. 0<=wei_freq<1.
        wei_freq == 1 implies that weights are sorted at each step.
        wei_freq == 0.1 implies that weights sorted each 10th step (faster,
            default value)
        wei_freq == 0 implies no sorting of weights (not recommended)
    seed : hashable, optional
        If None (default), use the np.random's global random state to generate random numbers.
        Otherwise, use a new np.random.RandomState instance seeded with the given value.
    Returns
    -------
    W0 : NxN np.ndarray
        randomized weighted connection matrix
    R : 4-tuple of floats
        Correlation coefficients between strength sequences of input and
        output connection matrices, rpos_in, rpos_out, rneg_in, rneg_out
    Notes
    -----
    The value of bin_swaps is ignored when binary topology is fully
        connected (e.g. when the network has no negative weights).
    Randomization may be better (and execution time will be slower) for
        higher values of bin_swaps and wei_freq. Higher values of bin_swaps
        may enable a more random binary organization, and higher values of
        wei_freq may enable a more accurate conservation of strength
        sequences.
    R are the correlation coefficients between positive and negative
        strength sequences of input and output connection matrices and are
        used to evaluate the accuracy with which strengths were preserved.
        Note that correlation coefficients may be a rough measure of
        strength-sequence accuracy and one could implement more formal tests
        (such as the Kolmogorov-Smirnov test) if desired.
    '''
    print("Hi")
    rng = get_rng(seed)
    if not np.all(W == W.T):
        raise BCTParamError("Input must be undirected")
    W = W.copy()
    n = len(W)
    np.fill_diagonal(W, 0)  # clear diagonal
    Ap = (W > 0)  # positive adjmat
    An = (W < 0)  # negative adjmat

    if np.size(np.where(Ap.flat)) < (n * (n - 1)):
        W_r, eff = randmio_und_signed(W, bin_swaps, seed=rng)
        Ap_r = W_r[0] > 0
        An_r = W_r[0] < 0
    else:
        Ap_r = Ap
        An_r = An
    print("O_o")
    W0 = np.zeros((n, n))
    for s in (1, -1):
        if s == 1:
            Acur = Ap
            A_rcur = Ap_r
        else:
            Acur = An
            A_rcur = An_r
        print("¯\_(ツ)_/¯")
        S = np.sum(W * Acur, axis=0)  # strengths
        Wv = np.sort(W[np.where(np.triu(Acur))])  # sorted weights vector
        i, j = np.where(np.triu(A_rcur))
        Lij, = np.where(np.triu(A_rcur).flat)  # weights indices

        P = np.outer(S, S)

        if wei_freq == 0:  # get indices of Lij that sort P
            Oind = np.argsort(P.flat[Lij])  # assign corresponding sorted
            W0.flat[Lij[Oind]] = s * Wv  # weight at this index
        else:
            wsize = np.size(Wv)
            wei_period = int(np.round(1 // wei_freq))  # convert frequency to period
            print(wei_period)
            lq = np.arange(wsize, 0, -wei_period, dtype=int)
            for m in lq:  # iteratively explore at this period
                # get indices of Lij that sort P
                Oind = np.argsort(P.flat[Lij])
                print("this is m")
                print(m)
                print("this is wei_period")
                print(wei_period)
                R = rng.permutation(m)[:np.min((m, wei_period))]
                print(R)
                try:
                    if len(R) > 1 and max(R) < max(Oind):
                        for q, r in enumerate(R):
                            print("enumerating....")
                            # choose random index of sorted expected weight
                            o = Oind[r]
                            W0.flat[Lij[o]] = s * Wv[r]  # assign corresponding weight

                            # readjust expected weighted probability for i[o],j[o]
                            f = 1 - Wv[r] / S[i[o]]
                            P[i[o], :] *= f
                            P[:, i[o]] *= f
                            f = 1 - Wv[r] / S[j[o]]
                            P[j[o], :] *= f
                            P[:, j[o]] *= f

                            # readjust strength of i[o]
                            S[i[o]] -= Wv[r]
                            # readjust strength of j[o]
                            S[j[o]] -= Wv[r]

                        O = Oind[R]
                        # remove current indices from further consideration
                        Lij = np.delete(Lij, O)
                        i = np.delete(i, O)
                        j = np.delete(j, O)
                        Wv = np.delete(Wv, R)
                    else:
                        continue
                except:
                    continue

    W0 = W0 + W0.T

    rpos_in = np.corrcoef(np.sum(W * (W > 0), axis=0),
                          np.sum(W0 * (W0 > 0), axis=0))
    rpos_ou = np.corrcoef(np.sum(W * (W > 0), axis=1),
                          np.sum(W0 * (W0 > 0), axis=1))
    rneg_in = np.corrcoef(np.sum(-W * (W < 0), axis=0),
                          np.sum(-W0 * (W0 < 0), axis=0))
    rneg_ou = np.corrcoef(np.sum(-W * (W < 0), axis=1),
                          np.sum(-W0 * (W0 < 0), axis=1))
    return W0, (rpos_in[0, 1], rpos_ou[0, 1], rneg_in[0, 1], rneg_ou[0, 1])


In [ ]:
z['nulls']={}
for keys, values in z['mu'].items():
    print(keys)
    nullz=null_model_und_sign(values, bin_swaps=5, wei_freq=.01)
    z['nulls'][keys]=nullz
   

sub-NDARINVMFVA1BZF
Hi


In [98]:
z['nulls'].keys()


dict_keys(['sub-NDARINVMFVA1BZF', 'sub-NDARINVD9L81NY5'])

## Check the partition
Will use normalized mutual information to assess similarity between network assignments. NMI measures information shared between two probability distribution functions, specifically measuring how much knowing one distribution leads to certainty ofthe other. Permuted the labels of individual matrices between contrasts 1,000 times to generate a null distribution of NMI values for each contrast. Matrices between groups were randomly shuffled and partitioned into functional networks, and NMI was calculated.   
https://scikit-learn.org/stable/modules/generated/sklearn.metrics.normalized_mutual_info_score.html#sklearn.metrics.normalized_mutual_info_score

### Normalized information score  
sklearn.metrics.normalized_mutual_info_score(labels_true [group 1], labels_pred [group 2], average_method=’warn’)

In [ ]:
diff_no_ov=normalized_mutual_info_score(norm_max, ov_max)

In [ ]:
diff_no_ob=normalized_mutual_info_score(norm_max, ob_max)

In [ ]:
diff_ov_ob=normalized_mutual_info_score(ov_max, ob_max)

## Parcelation 
Through BIAC https://wiki.biac.duke.edu/biac:analysis:resting_pipeline

## Individual and Group Matrices
Network-level analysis will be performed with inividual correltion matrices

## Thresholding
In accordance with van den Heuvel et al. 2017, we will examine and test statistical differences in functional connectivity (FC) defined as the mean of the correlation matrix. FC will be included in statistical tests between groups.

## Partitioning
Will partition full 264 connectome into modules using louvain algorithm. 

## Connectivity Strength
Caluclate Euclidean distance for each ROI-ROI pair. Linear regression with distance as a predictor of connectivity strength between groups.  
sklearn.metrics.pairwise.euclidean_distances(X, Y=None, Y_norm_squared=None, squared=False, X_norm_squared=None)

### Within network changes
All within network pairwise relationships were averaged per group. Two-tailed T-test to assess differences. Bonferroni corrections as needed.

### Between network changes
Average connectivity is calculated per network. Compare the between network interactions. 

## Pickling data to save it

In [ ]:
z['CC']= allCC
z['PC']= allPC
z.keys()

In [ ]:
for keys, values in z['edges'].items():
    print(values)
    values[0].reset_index().to_json()
    

In [ ]:
with open('currents.json', 'w', encoding='utf-8') as outfile:
    json.dump(z, outfile, ensure_ascii=False, indent=2)
